# Simulations of Survival rate

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

# Own Classes
from Simulation_Routing import *
from Simulating import *
from Simulation_Routing_Matrix import *
from Simulation_Routing_Matrix_copy import *
from Simulation_Routing_Matrix_Batch import *
from tqdm import tqdm

Initialize the class using the ip of the server.

In [2]:
ip = "54.93.239.5"

In [3]:
Simulations = Simulation(ip)

In [ ]:
RoutingMatrixCopy = RoutingSimulationMatrixBatch(ip)

In [ ]:
# Number of times the simulations are run to later get an average survivability rate
NO_SIMULATIONS = 6

# Likelihood of a responder declining the call to action
decline_rate = 0.7
# Maximal number of responder contacted
max_number_responder = 100
# Time of the incident
opening_hour = 12.0
# list of values that are in the available column
filter_values = ["Yes"]
# time of the day
time_of_day = "day" if 8.0 <= opening_hour <= 18.0 else "night"
# proportion of CFR of the total (ex.: 0.01 = 1%)
proportion = 0.001

df_final = pd.DataFrame()

x = 0
while x < NO_SIMULATIONS:
    try:
        df = Simulations.simulation_run(
            decline_rate, max_number_responder, opening_hour,
            filter_values, time_of_day, proportion
        )
        df['filter_values'] = filter_values
        df['opening_hour'] = opening_hour
        df['decline_rate'] = decline_rate
        df['max_number_responder'] = max_number_responder
        df['proportion_of_CFR'] = proportion
        # Rename all columns except 'patient_loc'
        suffix = f"_run{x+1}"
        df_renamed = df.rename(
            columns={col: f"{col}{suffix}" for col in df.columns if col != 'patient_loc'}
        )

        # Merge into the master DataFrame
        if df_final is None:
            df_final = df_renamed
        else:
            df_final = pd.merge(df_final, df_renamed, on="patient_loc", how="outer")

    except Exception as e:
        print(f"[Warning] Simulation {x+1} failed: {e}. Retrying...")

In [ ]:
# SAVE THE DATAFRAME!!!
df_final.to_csv("Your path here")